# Assignment 1

In [6]:
# install spacy and pandas in terminal, using "pip install spacy pandas"

In [7]:
import spacy
import pandas as pd
import os

/tmp/ipykernel_11126/2223173363.py:2: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [8]:
# install the model by typing in "python -m spacy download en_core_web_md" in the terminal

In [9]:
nlp = spacy.load("en_core_web_md")

In [10]:
# folderpath = os.path.join("../input/USEcorpus/a1")

In [11]:
main_folder_path = "../input/USEcorpus/"
sorted_dir = sorted(os.listdir(main_folder_path))

In [12]:
# with open(folder_path, "r", encoding="latin-1") as file:
    text=file.read()

IndentationError: unexpected indent (1164528371.py, line 2)

In [13]:
for folder in sorted_dir:
    folder_path = os.path.join(main_folder_path, folder)
    filenames = sorted(os.listdir(folder_path))
    for text_file in filenames:
        file_path = folder_path + "/" + text_file
        with open (file_path, encoding="latin-1") as file:
            text = file.read()
        doc = nlp(text)
       
        #load file
        #process with spacy
        #save results

KeyboardInterrupt: 

In [18]:
# create empty list
entities = []

# add each entity to list
for ent in doc.ents:
    entities.append(ent.text)
print(set(entities))

{'daily', 'four', 'New York', 'about nine years', 'Cosby', 'Welsh', 'English', 'Huxtable', 'Welshmen', 'Just two months ago', 'Swedish', 'today', 'one', 'last summer', 'Tom Cleese British', 'twelve', 'Dolly Parton'}


In [21]:
noun_count = 0
verb_count = 0
adj_count = 0
adv_count = 0

for token in doc:
    if token.pos_ == "NOUN":
        noun_count += 1
    elif token.pos_ == "VERB":
        verb_count += 1
    elif token.pos_ == "ADJ":
        adj_count += 1
    elif token.pos_ == "ADV":
        adv_count += 1

In [22]:
relative_freq_noun = (noun_count/len(doc)) * 10000
round(relative_freq_noun, 2)

1378.0

In [23]:
relative_freq_verb = (verb_count/len(doc)) * 10000
round(relative_freq_verb, 2)

1453.86

In [25]:
relative_freq_adj = (adj_count/len(doc)) * 10000
round(relative_freq_adj, 2)

518.33

In [26]:
relative_freq_adv = (adv_count/len(doc)) * 10000
round(relative_freq_adv, 2)

695.32

In [28]:
persons = set()
for ent in doc.ents:
        if ent.label_ == 'PERSON':
            persons.add(ent.text)

num_persons = len(persons)
print(num_persons)

5


In [29]:
locations = set()
for ent in doc.ents:
        if ent.label_ == 'LOC':
            locations.add(ent.text)

num_locations = len(locations)
print(num_locations)

0


In [30]:
organisations = set()
for ent in doc.ents:
        if ent.label_ == 'ORG':
            organisations.add(ent.text)

num_organisations = len(organisations)
print(num_organisations)

0


In [31]:
# create spaCy doc
# create a new Doc object
doc = nlp(text)

In [33]:
annotations = []
for token in doc:
    annotations.append((token.text, token.pos_))

In [35]:
# spaCy doc to pandas dataframe
data = pd.DataFrame(annotations, 
                    columns=["RelFreq NOUNS", "RelFreq VERB", "RelFreq ADJ", "RelFreq ADV", "Unique PER", "Unique LOC", "Unique ORG"])

ValueError: 7 columns passed, passed data had 2 columns

In [36]:
# extra line